In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
t0 = time.time()
# total_k_lines = 100
# dutch_data=pd.read_csv("Dutch_Updated.txt",sep='\n',header=None,skiprows=(2027358-total_k_lines*1000), nrows=total_k_lines*1000)
# english_data=pd.read_csv("English_Updated.txt",sep='\n',header=None,skiprows=(2027358-total_k_lines*1000), nrows=total_k_lines*1000)

In [3]:
# FOR TRAINING FROM DUTCH TO ENGLISH
dutch_data=pd.read_csv("Dutch_test_1000.txt",sep='\n',header=None)
english_data=pd.read_csv("English_test_1000.txt",sep='\n',header=None)
df = pd.DataFrame(columns=['dutch','english'])

In [4]:
# FOR TRAINING FROM ENGLISH TO DUTCH
# dutch_data=pd.read_csv("English_Training.txt",sep='\n',header=None)
# english_data=pd.read_csv("Dutch_Training.txt",sep='\n',header=None)
# df = pd.DataFrame(columns=['english','dutch'])

for a in range(len(english_data)):
        df.loc[a] = dutch_data[0][a].split(" "),english_data[0][a].split(" ")

del english_data
del dutch_data

eng_dut_pair = df.to_numpy()
del df

In [5]:
# Extract dutch and english vocabularies
def vocab(eng_dut_pair):
    dutch_words = []
    english_words = []

    for s_p in eng_dut_pair:
        for ew in s_p[1]: 
            english_words.append(ew)
        for fw in s_p[0]: 
            dutch_words.append(fw)

    english_words = sorted(list(set(english_words)), key=lambda s: s.lower()) 
    dutch_words = sorted(list(set(dutch_words)), key=lambda s: s.lower())
    # print('English vocab: ', english_words)
    # print('dutch vocab: ',dutch_words)
    return english_words,dutch_words



In [6]:
# Vocabulary size of the english and dutch words
def vocab_size(english_words,dutch_words):
    english_vocab_size = len(english_words)
    dutch_vocab_size = len(dutch_words)
    print('english_vocab_size: ', english_vocab_size)
    print('dutch_vocab_size: ', dutch_vocab_size)
    return english_vocab_size,dutch_vocab_size



In [7]:
# Routine to uniformly initialize word translation probabilities in t hash
def init_prob(t, prev, init_val, eng_dut_pair):
    for a in range(len(eng_dut_pair)):
        for d_word in eng_dut_pair[a][0]:
            for e_word in eng_dut_pair[a][1]:
                t[(e_word, d_word)] = init_val
                prev[(e_word, d_word)] = 0


# Main routine
english_words,dutch_words = vocab(eng_dut_pair)
english_vocab_size,dutch_vocab_size = vocab_size(english_words,dutch_words)



english_vocab_size:  846
dutch_vocab_size:  863


In [8]:
# Initialize probabilities uniformly
final_total = {}
t = {}
prev = {}
init_val = 1.0 / dutch_vocab_size
init_prob(t, prev, init_val, eng_dut_pair)

print('No. of dutch/english pairs: ', len(t))


delta = 1 
epsilon = 0.02

number_of_epochs = 0
# Loop while not converged
while (delta > epsilon):

    # Initialize
    count = {}
    total = {}
    pair_delta = 0
    
    for sp in eng_dut_pair:
        for fw in sp[0]:
            total[fw] = 0.0
            for ew in sp[1]:
                count[(ew,fw)] = 0.0
        
    for sp in eng_dut_pair:

        # Compute normalization
        for ew in sp[1]:
            final_total[ew] = 0.0
            for fw in sp[0]:
                final_total[ew] += t[(ew, fw)]

        # Collect counts
        for ew in sp[1]:
            for fw in sp[0]:
                count[(ew, fw)] += t[(ew, fw)] / final_total[ew]
                total[fw] += t[(ew, fw)] / final_total[ew]

                
    for a in range(len(eng_dut_pair)):
        for e_word in eng_dut_pair[a][1]:
            for d_word in eng_dut_pair[a][0]:
                pair_delta = max(pair_delta, abs(prev[(e_word, d_word)]-t[(e_word, d_word)]))
                prev[(e_word, d_word)] = t[(e_word, d_word)]
                t[(e_word, d_word)] = count[(e_word, d_word)] / total[d_word]
                
    
    number_of_epochs +=1
    delta = pair_delta
    print(delta)
    if(number_of_epochs>20):
        break

print("number_of_epochs",number_of_epochs)

# np.save('Trained_weights.npy', t)

#Finding the maximum matrix
print("No. of dutch/english pairs:",len(t))

convert = {}
coverted_prob = {}

for p_ed in t.keys():
    if p_ed[1] not in coverted_prob.keys():
        coverted_prob[p_ed[1]] = 0
    if(coverted_prob[p_ed[1]] < t[p_ed]):
        coverted_prob[p_ed[1]] = t[p_ed]
        convert[p_ed[1]] = p_ed[0]

np.save('Trained_mapping.npy',convert)

t1 = time.time()
print("Time taken to train the model",time.strftime('%H:%M:%S', time.gmtime(t1-t0)))

No. of dutch/english pairs:  44559
0.18634125144843572
0.2488412514484357
0.24732187178698567
0.1951111342649478
0.18615645687166066
0.14218162559752567
0.10439969734404647
0.10482088066128292
0.09715745722770464
0.08345243095140809
0.06832976207467056
0.05441326436972238
0.048863723302862705
0.043278102204199476
0.037953677484295345
0.03312991472303772
0.028884830288305796
0.025208334131250343
0.02204954806148174
0.020421244044887016
0.02096573816012437
number_of_epochs 21
No. of dutch/english pairs: 44559
Time taken to train the model 00:00:06
